In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

haiku_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class Haiku writer who is good at capturing the characteristics of things and abstracting them.",
        ),
        ("human", "I want to get a haiku about program language {lang}."),
    ]
)

haiku_chain = haiku_prompt | chat

explain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert at interpreting and explaining haiku well.",
        ),
        ("human", "{haiku}"),
    ]
)


explain_chain = explain_prompt | chat


final_chain = {"haiku": haiku_chain} | explain_chain

final_chain.invoke({"lang": "C#"})

Elegant syntax flows,
C# dances with logic's code,
Crafting worlds in lines.This haiku beautifully captures the essence of programming in C#. The first line sets the tone with "Elegant syntax flows," suggesting a sense of grace and precision in writing code. The second line, "C# dances with logic's code," personifies the programming language as it interacts with the logical structures of a program. Finally, the last line, "Crafting worlds in lines," speaks to the power of coding to create entire virtual worlds through lines of code. Overall, this haiku celebrates the artistry and creativity involved in programming with C#.

AIMessageChunk(content='This haiku beautifully captures the essence of programming in C#. The first line sets the tone with "Elegant syntax flows," suggesting a sense of grace and precision in writing code. The second line, "C# dances with logic\'s code," personifies the programming language as it interacts with the logical structures of a program. Finally, the last line, "Crafting worlds in lines," speaks to the power of coding to create entire virtual worlds through lines of code. Overall, this haiku celebrates the artistry and creativity involved in programming with C#.')